# 2. Парсинг результатов поименного голосования

Установка необходимых пакетов

In [1]:
!pip install Scrapy==1.6.0
!pip install Twisted==19.2.0

  Using cached Scrapy-1.6.0-py2.py3-none-any.whl (231 kB)
     |████████████████████████████████| 54 kB 306 kB/s eta 0:00:01
  Using cached Twisted-20.3.0.tar.bz2 (3.1 MB)
  Using cached service_identity-18.1.0-py2.py3-none-any.whl (11 kB)
  Using cached w3lib-1.22.0-py2.py3-none-any.whl (20 kB)
  Using cached queuelib-1.5.0-py2.py3-none-any.whl (13 kB)
     |████████████████████████████████| 5.4 MB 2.0 MB/s eta 0:00:01
  Using cached PyDispatcher-2.0.5.tar.gz (34 kB)
     |████████████████████████████████| 2.6 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 158 kB/s eta 0:00:01
  Using cached constantly-15.1.0-py2.py3-none-any.whl (7.9 kB)
     |████████████████████████████████| 48 kB 676 kB/s eta 0:00:01
  Using cached incremental-17.5.0-py2.py3-none-any.whl (16 kB)
  Using cached zope.interface-5.2.0-cp38-cp38-manylinux2010_x86_64.whl (244 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 

In [7]:
import scrapy
import os
import datetime
import json
from asyncio import Queue
from multiprocessing import Process
from scrapy import crawler
from twisted.internet import reactor

from scrapy.crawler import CrawlerProcess

Парсить результаты будем из файла с п.1

Крайне простой парсер из системы w1.c1.rada.gov.ua, который полагается (на текущий момент) захардкоженные айди блоков для каждой партии/депутатской фракции

In [2]:
class DeputyVotingSpider(scrapy.Spider):
    name = "articles"
    start_urls = [
        "http://w1.c1.rada.gov.ua/pls/radan_gs09/ns_golos?g_id=5900"
    ]

    def parse(self, response):
        sn_frac = response.css("#0idf1")
        for li in sn_frac.css('ul.frd').css('li'):
            answer = {
                "party": "СН",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        opzg_frac = response.css("#0idf2")
        for li in opzg_frac.css('ul.frd').css('li'):
            answer = {
                "party": "ОПЗЖ",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        es_frac = response.css("#0idf4")
        for li in es_frac.css('ul.frd').css('li'):
            answer = {
                "party": "ЕС",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        btk_frac = response.css("#0idf3")
        for li in btk_frac.css('ul.frd').css('li'):
            answer = {
                "party": "Батьківщина",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        # some bug in system?? 
        # для ЗаМайбаха есть два разных айди на разных страницах (#0idf8 == #0idf6)

        zm_frac = response.css("#0idf8")
        for li in zm_frac.css('ul.frd').css('li'):
            answer = {
                "party": "За Майбутне",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        zm_frac = response.css("#0idf6")
        for li in zm_frac.css('ul.frd').css('li'):
            answer = {
                "party": "За Майбутне",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        golos_frac = response.css("#0idf5")
        for li in golos_frac.css('ul.frd').css('li'):
            answer = {
                "party": "Голос",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        dovira_frac = response.css("#0idf7")
        for li in dovira_frac.css('ul.frd').css('li'):
            answer = {
                "party": "Довіра",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

        zm_frac = response.css("#0idf0")
        for li in zm_frac.css('ul.frd').css('li'):
            answer = {
                "party": "Позафракційні",
                "name": "".join(li.css('div.dep::text').extract()).strip(),
                "result": "".join(li.css('div.golos')[0].css('::text').extract()).strip()
            }
            yield answer

In [3]:
# by default, CrawlerProcess is not restartable, so here it is a wrapper to make it run more times

def run_spider(spider, json_file):
    def f(q):
        try:
            runner = crawler.CrawlerRunner({
                'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
                'FEED_FORMAT': 'json',
                'FEED_URI': json_file
            })
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        return result

Для удобства, создадим каталог для каждого конкретного голосования

In [9]:
all_laws_file = open("vr_session_parse.json", "r")
text = all_laws_file.read()
json_info = json.loads(text)
all_laws_file.close()

In [10]:
try:
    os.mkdir("deputy_vote_result")
except FileExistsError:
    pass

In [11]:
for item in json_info:
    link = item['link']
    law_id = item['id']

    JSON_FILE = f"./deputy_vote_result/{law_id}/result.json"

    try:
        os.mkdir(f'./deputy_vote_result/{law_id}')
        DeputyVotingSpider.start_urls = [f"http://w1.c1.rada.gov.ua/pls/radan_gs09/ns_golos?g_id={law_id}"]
        run_spider(DeputyVotingSpider, JSON_FILE)

        f_1 = open(JSON_FILE, "r+")

        text = f_1.read()
        f_1.seek(0)
        json_info = json.loads(text)
        new_text = json.dumps(json_info, ensure_ascii=False, indent=1)

        f_1.write(new_text)
        f_1.truncate()
        f_1.close()
    except FileExistsError:
        pass

<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-11-43be37ac17c8>:10: RuntimeWarning: coroutine 'Queue.get' was never awaited
  run_spider(DeputyVotingSpider, JSON_FILE)
<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-3-ca3bc30c0504>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)
<ipython-input-3-ca3bc30

KeyboardInterrupt: 